The following tutorials are available from the [Wallaroo Tutorials Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-model-deploy-and-serve/notebooks_in_prod).

# Stage 4: Regular Batch Inference

In Stage 3: Deploy the Model in Wallaroo, the housing model created and tested in Stage 2: Training Process Automation Setup was uploaded to a Wallaroo instance and added to the pipeline `housing-pipe` in the workspace `housepricing`.  This pipeline can be deployed at any point and time and used with new inferences.

For the purposes of this demo, let's say that every month we find the newly entered and still-unsold houses and predict their sale price.

The predictions are entered into a staging table, for further inspection before being joined to the primary housing data table.

We show this as a notebook, but this can also be scripted and scheduled, using CRON or some other process.

## Resources

The following resources are used as part of this tutorial:

* **data**
  * `data/seattle_housing_col_description.txt`: Describes the columns used as part data analysis.
  * `data/seattle_housing.csv`: Sample data of the Seattle, Washington housing market between 2014 and 2015.
* **code**
  * `simdb.py`: A simulated database to demonstrate sending and receiving queries.
* **models**
  * `housing_model_xgb.onnx`: Model created in Stage 2: Training Process Automation Setup.
  * `./models/preprocess_step.zip.`: Formats the incoming data for the model.
  * `./models/postprocess_step.zip`: Formats the outgoing data for the model.


## Steps

This process will use the following steps:

* [Connect to Wallaroo](#connect-to-wallaroo): Connect to the Wallaroo instance and the `housepricing` workspace.
* [Deploy the Pipeline](#deploy-the-pipeline): Deploy the pipeline to prepare it to run inferences.
* [Read In New House Listings](#read-in-new-house-listings): Read in the previous month's house listings and submit them to the pipeline for inference.
* [Send Predictions to Results Staging Table](#send-predictions-to-results-staging-table): Add the inference results to the results staging table.

### Connect to Wallaroo

Connect to the Wallaroo instance and set the `housepricing` workspace as the current workspace.

In [1]:
import json
import pickle
import wallaroo
import pandas as pd
import numpy as np
import pyarrow as pa
import datetime

import simdb # module for the purpose of this demo to simulate pulling data from a database

from wallaroo_client import get_workspace

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)


### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

In [3]:
workspace_name = 'housepricing'
model_name = "housepricemodel"
model_file = "./housing_model_xgb.onnx"
pipeline_name = "housing-pipe"

In [4]:
new_workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
_ = wl.set_current_workspace(new_workspace)

### Deploy the Pipeline

Deploy the `housing-pipe` workspace established in Stage 3: Deploy the Model in Wallaroo (`03_deploy_model.ipynb`).

In [5]:
new_workspace.pipelines()

[{'name': 'housing-pipe', 'create_time': datetime.datetime(2024, 4, 8, 18, 10, 48, 940254, tzinfo=tzutc()), 'definition': '[]'}]

In [6]:
pipeline = wl.get_pipeline(pipeline_name)
pipeline

name,housing-pipe
created,2024-04-08 18:10:48.940254+00:00
last_updated,2024-04-11 19:51:23.423449+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"49dce46a-3cc8-44d7-b1f8-2fe2b1ef4919, 6709af7a-f6cc-40f1-a0f6-e43912d1e308, 682ab64c-239d-4cda-9f84-d8395b9747b9, dea96d38-f411-481e-a719-e4e307c5f51b, 34926083-1324-45b3-bded-36bec313bd46, 7b730932-9462-4d49-b42c-80b1996d5707, 2497971a-6c59-42c3-ac54-09917308be6a, 902bd20a-b57e-4dae-a045-df14013a33f0, e7da71eb-8642-4379-ae38-e2e4f57705e4, bb3dc349-1709-45ac-8c6f-2734838125d5"
steps,housepricemodel
published,False


In [7]:
pipeline.undeploy()
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(0.5).memory("1Gi").build()
pipeline.deploy(deployment_config=deploy_config)

name,housing-pipe
created,2024-04-08 18:10:48.940254+00:00
last_updated,2024-04-11 19:54:44.941537+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"0154edb5-3c2b-44fd-b7a7-1c1d3705c32d, 49dce46a-3cc8-44d7-b1f8-2fe2b1ef4919, 6709af7a-f6cc-40f1-a0f6-e43912d1e308, 682ab64c-239d-4cda-9f84-d8395b9747b9, dea96d38-f411-481e-a719-e4e307c5f51b, 34926083-1324-45b3-bded-36bec313bd46, 7b730932-9462-4d49-b42c-80b1996d5707, 2497971a-6c59-42c3-ac54-09917308be6a, 902bd20a-b57e-4dae-a045-df14013a33f0, e7da71eb-8642-4379-ae38-e2e4f57705e4, bb3dc349-1709-45ac-8c6f-2734838125d5"
steps,housepricemodel
published,False


### Read In New House Listings

From the data store, load the previous month's house listing, prepare it as a DataFrame, then submit it for inferencing.

In [8]:
conn = simdb.simulate_db_connection()

# create the query
query = f"select * from {simdb.tablename} where date > DATE(DATE(), '-1 month') AND sale_price is NULL"
print(query)

# read in the data
# can't have null values - turn them into 0
newbatch = pd.read_sql_query(query, conn)
newbatch['sale_price'] = newbatch.sale_price.apply(lambda x: 0)
display(newbatch.shape)
display(newbatch.head(10).loc[:, ["id", "date", "list_price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot"]])

select * from house_listings where date > DATE(DATE(), '-1 month') AND sale_price is NULL


(1090, 22)

,id,date,list_price,bedrooms,bathrooms,sqft_living,sqft_lot
0,9215400105,2024-03-13,450000.0,3,1.75,1250,5963
1,1695900060,2024-03-26,535000.0,4,1.00,1610,2982
2,9545240070,2024-03-13,660500.0,4,2.25,2010,9603
3,1432900240,2024-03-23,205000.0,3,1.00,1610,8579
4,6131600075,2024-03-12,225000.0,3,1.00,1300,8316
5,1400300055,2024-03-13,425000.0,2,1.00,770,5040
6,7960900060,2024-03-19,2900000.0,4,3.25,5050,20100
7,6378500125,2024-03-16,436000.0,2,1.00,1040,7538
8,2022069200,2024-03-20,455000.0,4,2.50,2210,49375
9,9412900055,2024-03-20,405000.0,3,1.75,2390,6000


In [9]:
# query = {'query': newbatch.to_json()}

result = pipeline.infer(newbatch)
display(result)

,time,in.bathrooms,in.bedrooms,in.condition,in.date,in.floors,in.grade,in.id,in.lat,in.list_price,...,in.sqft_living15,in.sqft_lot,in.sqft_lot15,in.view,in.waterfront,in.yr_built,in.yr_renovated,in.zipcode,out.variable,anomaly.count
0,2024-04-11 19:55:12.776,1.75,3,4,2024-03-13,1.0,7,9215400105,47.6796,450000.0,...,970,5963,5100,0,0,1953,0,98115,[508255.0],0
1,2024-04-11 19:55:12.776,1.00,4,4,2024-03-26,1.5,7,1695900060,47.5870,535000.0,...,1610,2982,4040,0,0,1925,0,98144,[500198.0],0
2,2024-04-11 19:55:12.776,2.25,4,3,2024-03-13,1.0,8,9545240070,47.5343,660500.0,...,2060,9603,9793,0,0,1986,0,98027,[539598.0],0
3,2024-04-11 19:55:12.776,1.00,3,4,2024-03-23,1.0,7,1432900240,47.4563,205000.0,...,1610,8579,8579,0,0,1962,0,98058,[270739.0],0
4,2024-04-11 19:55:12.776,1.00,3,4,2024-03-12,1.0,6,6131600075,47.3221,225000.0,...,1260,8316,8316,0,0,1954,0,98002,[191304.0],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,2024-04-11 19:55:12.776,2.75,4,3,2024-03-22,2.0,9,3304300300,47.4828,579950.0,...,3110,8643,8626,0,0,2011,0,98059,[577492.0],0
1086,2024-04-11 19:55:12.776,2.50,4,3,2024-03-20,2.0,10,6453550090,47.6060,861111.0,...,3860,7090,7272,0,0,2008,0,98074,[882930.0],0
1087,2024-04-11 19:55:12.776,2.25,3,3,2024-03-13,2.0,7,1760650820,47.3589,290000.0,...,1610,3764,3825,0,0,2012,0,98042,[271484.0],0
1088,2024-04-11 19:55:12.776,3.50,4,3,2024-03-17,2.0,8,3345700207,47.5252,608500.0,...,2850,5577,5708,0,0,2014,0,98056,[537434.0],0


In [10]:
predicted_prices = pd.DataFrame(result['out.variable'].apply(lambda x: x[0])).rename(columns={'out.variable':'prediction'})
display(predicted_prices[0:5])

,prediction
0,508255.0
1,500198.0
2,539598.0
3,270739.0
4,191304.0


### Send Predictions to Results Staging Table

Take the predicted prices based on the inference results so they can be joined into the `house_listings` table.

Once complete, undeploy the pipeline to return the resources back to the Kubernetes environment.

In [11]:
result_table = pd.DataFrame({
    'id': newbatch['id'],
    'saleprice_estimate': predicted_prices['prediction']
})

display(result_table)

result_table.to_sql('results_table', conn, index=False, if_exists='append')

,id,saleprice_estimate
0,9215400105,508255.0
1,1695900060,500198.0
2,9545240070,539598.0
3,1432900240,270739.0
4,6131600075,191304.0
...,...,...
1085,3304300300,577492.0
1086,6453550090,882930.0
1087,1760650820,271484.0
1088,3345700207,537434.0


In [12]:
# Display the top of the table for confirmation
pd.read_sql_query("select * from results_table limit 5", conn)

,id,saleprice_estimate
0,9215400105,508255.0
1,1695900060,500198.0
2,9545240070,539598.0
3,1432900240,270739.0
4,6131600075,191304.0


In [13]:
conn.close()
pipeline.undeploy()

name,housing-pipe
created,2024-04-08 18:10:48.940254+00:00
last_updated,2024-04-11 19:54:44.941537+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"0154edb5-3c2b-44fd-b7a7-1c1d3705c32d, 49dce46a-3cc8-44d7-b1f8-2fe2b1ef4919, 6709af7a-f6cc-40f1-a0f6-e43912d1e308, 682ab64c-239d-4cda-9f84-d8395b9747b9, dea96d38-f411-481e-a719-e4e307c5f51b, 34926083-1324-45b3-bded-36bec313bd46, 7b730932-9462-4d49-b42c-80b1996d5707, 2497971a-6c59-42c3-ac54-09917308be6a, 902bd20a-b57e-4dae-a045-df14013a33f0, e7da71eb-8642-4379-ae38-e2e4f57705e4, bb3dc349-1709-45ac-8c6f-2734838125d5"
steps,housepricemodel
published,False


From here, organizations can automate this process.  Other features could be used such as data analysis using Wallaroo assays, or other features such as shadow deployments to test champion and challenger models to find which models provide the best results.